In [ ]:
import numpy as np
import os
import pandas as pd
from scipy.spatial import distance

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import utils.const as const
from pymongo import MongoClient
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
filename='text_post.csv'
db=const.grpId
collection=const.PostCollection

In [ ]:
client = MongoClient()
myCollect=client[db][collection]

In [ ]:
df_test = pd.read_csv(filename, nrows=1, header=None)
featureCols = []
float32_cols = {}
for c in df_test:
  if c==0:
    continue
  
  featureCols.append(c)
  float32_cols[c] = np.float32

In [ ]:
df = pd.read_csv(filename, engine='c', dtype=float32_cols,header=None) 

In [ ]:
df.head()

# visualisation  using PCA

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(df[featureCols].values)

print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=pca_result[:,0], 
    ys=pca_result[:,1], 
    zs=pca_result[:,2],
    cmap='tab10'
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()

# printing K nearest images using cosine similarity

In [ ]:
A=np.array(df[featureCols])

In [ ]:
#postId = '1991716644473195_2795420990769419'
postId = df[0].values[0]

index=df.index[df[0] == postId].tolist()[0]

neighCount=10

In [ ]:
distances = distance.cdist([A[index]], A, "cosine")[0]
indexes = np.argsort(distances)[:neighCount+1]

In [ ]:
def showText(postId):
    result = myCollect.find({'_id':postId})

    for x in result:
        if 'message' in x:
            print(x['message'])
        else:
            print("<<blank>>")

In [ ]:
print('File==>')
showText(df[0].values[index])

In [ ]:
print(str(neighCount)+' closest files')

for i in indexes:
  if i == index:
    continue
  print(df[0].values[i],distances[i])
  showText(df[0].values[i])
  